# Data Analysis

### In this section, we will clean and analyze our data to find insight that we can then visualize

### Now, we can begin to clean our data

In [3]:
# first we load our .csv
import pandas as pd

reddit_df = pd.read_csv("RedditWineData.csv")

# Let us take a look at our data
reddit_df.head()

,subreddit,title,selftext,upvote_ratio,ups,downs,score,created_utc,id,kind
0,r/wine,Champagne and In-N-Out to celebrate becoming a...,NaN,0.97,2127.0,0.0,2127.0,2021-03-03 18:30:23,lxaj0v,t3
1,r/wine,"Political, but thought you all would appreciat...",NaN,0.97,1988.0,0.0,1988.0,2020-11-11 05:12:09,js7vcf,t3
2,r/wine,"If you’re having a bad day, just know that at ...",NaN,0.99,1569.0,0.0,1569.0,2020-01-10 07:19:50,emsc51,t3
3,r/wine,"Hadn't seen this here, and it made me laugh.",NaN,0.99,1512.0,0.0,1512.0,2019-09-03 07:31:43,cz5n1v,t3
4,r/wine,Wine cellars are probably my favorite thing to...,NaN,0.99,1458.0,0.0,1458.0,2020-11-18 17:05:14,jwst3i,t3


In [4]:
# Check for any NaN values (besides selftext)
nan_df = reddit_df[reddit_df['title'].isna()]
nan_df.head()

,subreddit,title,selftext,upvote_ratio,ups,downs,score,created_utc,id,kind


#### We see that there are no rows returned, we can assume that there aren't any rows that are fully NaN

#### Now we need to change all titles and selftext into lowercase so that our string matching works in all instances.

In [5]:
reddit_df['title'] = reddit_df['title'].transform(lambda x: x.lower())
reddit_df['selftext'] = reddit_df['selftext'].transform(lambda x: x.lower() if isinstance(x,str) else x)

# Check to make sure our columns are in lowercase
temp = reddit_df[reddit_df['selftext'].notnull()]
print(temp.head())

    subreddit                                              title  \
72     r/wine                       any other winos on a budget?   
74     r/wine  remember: it's only "quarantine" if it's from ...   
164    r/wine        "why don't you have any of the good wines?"   
191    r/wine  how to be pretentious about wine? the perfect ...   
220    r/wine  reminder: don’t be a snob about the bottles pe...   

                                              selftext  upvote_ratio    ups  \
72   edit 3: *r/proletarianwine is now a thing!* th...          0.98  654.0   
74           otherwise, it's just sparkling isolation.          0.93  648.0   
164  i'm a manager at a giant liquor store. it's ab...          0.98  504.0   
191  answer by kalani tom, creator of the wine code...          0.97  468.0   
220  many of us are more knowledgeable about wine t...          0.97  454.0   

     downs  score          created_utc      id kind  
72     0.0  654.0  2021-02-09 08:22:38  lg6wr5   t3  
74     0

#### Here we will load our wine flavor dataset compiled from data from [these](https://media.winefolly.com/wine-flavor-profiles-red-wine.png#large) [infographics](https://media.winefolly.com/white-wine-flavor-profiles-folly.png#large) by [winefolly](www.winefolly.com)

In [6]:
# Create a DataFrame of common wines and their categories and attributes
# Remember to thank WineFolly.com for the wine flavor profiles
wines_df = pd.read_excel('WineFlavors.xlsx')
print(wines_df.head())

            Wine Name Classification  \
0            Albarino          White   
1             Barbera            Red   
2      Cabernet Franc            Red   
3  Cabernet Sauvignon            Red   
4           Carmenere            Red   

                                             Flavors  Body  Red Fruit  \
0  Lemon Zest, Grapefruit, Honeydew, Nectarine, S...   2.0        0.0   
1  Tart Cherry, Licorice, Blackberry, Dried Herbs...   3.0        5.0   
2  Strawberry, Raspberry, Bell Pepper, Crushed Gr...   3.0        5.0   
3  Black Cherry, Black Currant, Cedar, Baking Spi...   5.0        3.0   
4  Raspberry, Bell Pepper, Black Plum, Paprika, V...   3.0        5.0   

   Black Fruit  Floral  Herbal  Pepper  Earth  Baking Spice  Leather  \
0          0.0     3.0     0.0     0.0    0.0           0.0      0.0   
1          2.0     1.0     5.0     3.0    5.0           3.0      2.0   
2          0.0     2.0     5.0     2.0    5.0           1.0      0.0   
3          5.0     2.0     4.0  

#### Now we will create a column of wines mentioned in the post's title and selftext

In [7]:
def get_wine_mentions(wine_names, text):
    if isinstance(text, str):
        res = [wine for wine in wine_names if (wine.lower() in text)]
    else:
        res = []
    if res:
        return res
    else:
        return None

# Create a list of wine names from our WineFlavors dataset
wine_names = wines_df['Wine Name']

# Search our titles and selftext columns of each post for any mentions of wines in our wine_names list
reddit_df['wines_mentioned_in_title'] = reddit_df['title'].transform(lambda x: get_wine_mentions(wine_names, x))
reddit_df['wines_mentioned_in_selftext'] = reddit_df['selftext'].transform(lambda x: get_wine_mentions(wine_names, x))

reddit_df.head()

,subreddit,title,selftext,upvote_ratio,ups,downs,score,created_utc,id,kind,wines_mentioned_in_title,wines_mentioned_in_selftext
0,r/wine,champagne and in-n-out to celebrate becoming a...,NaN,0.97,2127.0,0.0,2127.0,2021-03-03 18:30:23,lxaj0v,t3,[Champagne],None
1,r/wine,"political, but thought you all would appreciat...",NaN,0.97,1988.0,0.0,1988.0,2020-11-11 05:12:09,js7vcf,t3,None,None
2,r/wine,"if you’re having a bad day, just know that at ...",NaN,0.99,1569.0,0.0,1569.0,2020-01-10 07:19:50,emsc51,t3,None,None
3,r/wine,"hadn't seen this here, and it made me laugh.",NaN,0.99,1512.0,0.0,1512.0,2019-09-03 07:31:43,cz5n1v,t3,None,None
4,r/wine,wine cellars are probably my favorite thing to...,NaN,0.99,1458.0,0.0,1458.0,2020-11-18 17:05:14,jwst3i,t3,None,None


In [8]:
# Combine the two columns: wines_mentioned_in_title and wines_mentioned_in_selftext into one column, total_mentions
def combine_lists(list1, list2):
    if isinstance(list1, list) and isinstance(list2, list):
        return list1.append(list2).unique()
    elif isinstance(list1, list) and not isinstance(list2, list):
        return list1
    elif isinstance(list2, list) and not isinstance(list1, list): 
        return list2
    else:
        return None

reddit_df['total_mentions'] = reddit_df.apply(lambda row: combine_lists(row['wines_mentioned_in_title'], row['wines_mentioned_in_selftext']), axis=1)
reddit_df[reddit_df['total_mentions'].notna()].head()

,subreddit,title,selftext,upvote_ratio,ups,downs,score,created_utc,id,kind,wines_mentioned_in_title,wines_mentioned_in_selftext,total_mentions
0,r/wine,champagne and in-n-out to celebrate becoming a...,NaN,0.97,2127.0,0.0,2127.0,2021-03-03 18:30:23,lxaj0v,t3,[Champagne],None,[Champagne]
22,r/wine,i'm an artist who just sold my *first* paintin...,NaN,0.99,899.0,0.0,899.0,2021-04-09 14:01:38,mnqlf7,t3,[Champagne],None,[Champagne]
30,r/wine,"yellow taill shiraz, my late fathers wine of c...",NaN,0.94,799.0,0.0,799.0,2020-09-20 17:34:43,iwq00f,t3,[Shiraz],None,[Shiraz]
51,r/wine,getting in on that champagne and fried chicken!,NaN,0.95,729.0,0.0,729.0,2021-04-01 05:08:53,mhtw9m,t3,[Champagne],None,[Champagne]
53,r/wine,1999 charles shaw cabernet sauvignon (tasting ...,NaN,0.99,726.0,0.0,726.0,2020-01-06 19:44:14,el5qkr,t3,[Cabernet Sauvignon],None,[Cabernet Sauvignon]


### Now that we have a list of total wine mentions in our data, we can begin using this information to find insights.

#### First, we will find the most mentioned wines in our data

In [9]:
# Now we count the wines mentioned by name
from collections import Counter

count = Counter()

for row in reddit_df['total_mentions']:
    count.update(Counter(row))

print(count)

Counter({'Champagne': 32, 'Riesling': 19, 'Pinot Noir': 15, 'Cabernet Sauvignon': 10, 'Chardonnay': 8, 'Merlot': 7, 'Shiraz': 6, 'Syrah': 6, 'Prosecco': 4, 'Malbec': 3, 'Pinot Gris': 3, 'Gewurztraminer': 3, 'Zinfandel': 2, 'Chenin Blanc': 2, 'Chablis': 2, 'Barbera': 2, 'Tempranillo': 2, 'Cabernet Franc': 1, 'Carmenere': 1, 'Monastrell': 1, 'Petite Sirah': 1, 'Sauvignon Blanc': 1, 'Viognier': 1, 'Montepulciano': 1, 'Albarino': 1, 'Vermentino': 1})


#### From these results, we can see that champagne, riesling, pinot noir, and cabernet sauvignon are the 4 most commonly mentioned wines in these posts

#### Now we will turn our wine counts into a DataFrame for further analysis and visualization

In [10]:
# We turn the dictionary generated from counting the mentioned wines into a DataFrame for later use
wine_counts = pd.DataFrame()
wine_counts = wine_counts.from_dict(count, orient='index')
wine_counts = wine_counts.reset_index()
wine_counts.columns = ['Wine Name', 'Count']
print(wine_counts)

# We will save the wine_counts DataFrame as .csv and .xlsx files for visualization and to keep for later if needed
wine_counts.to_csv("Wine_Counts.csv")
wine_counts.to_excel("Wine_Counts.xlsx")

             Wine Name  Count
0            Champagne     32
1               Shiraz      6
2   Cabernet Sauvignon     10
3             Riesling     19
4           Pinot Noir     15
5           Chardonnay      8
6               Merlot      7
7            Zinfandel      2
8         Chenin Blanc      2
9              Chablis      2
10             Barbera      2
11      Cabernet Franc      1
12           Carmenere      1
13              Malbec      3
14          Monastrell      1
15        Petite Sirah      1
16            Prosecco      4
17     Sauvignon Blanc      1
18               Syrah      6
19            Viognier      1
20         Tempranillo      2
21          Pinot Gris      3
22       Montepulciano      1
23      Gewurztraminer      3
24            Albarino      1
25          Vermentino      1


#### Here is a visual result using this data: 

![title](./Img/MostMentionedWines.png)

#### Here we merge the reddit_df and wines_df DataFrames together joining on the Wine Name

In [11]:
wines_df = wines_df.merge(wine_counts)
wines_df = wines_df.fillna(0)

wines_df.head()

,Wine Name,Classification,Flavors,Body,Red Fruit,Black Fruit,Floral,Herbal,Pepper,Earth,...,Leather,Astringency,Citrus Fruit,Stone Fruit,Tropical Fruit,Honey,Cream,Minerality,Bitter,Count
0,Albarino,White,"Lemon Zest, Grapefruit, Honeydew, Nectarine, S...",2.0,0.0,0.0,3.0,0.0,0.0,0.0,...,0.0,0.0,5.0,2.0,0.0,1.0,1.0,4.0,0.0,1
1,Barbera,Red,"Tart Cherry, Licorice, Blackberry, Dried Herbs...",3.0,5.0,2.0,1.0,5.0,3.0,5.0,...,2.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
2,Cabernet Franc,Red,"Strawberry, Raspberry, Bell Pepper, Crushed Gr...",3.0,5.0,0.0,2.0,5.0,2.0,5.0,...,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
3,Cabernet Sauvignon,Red,"Black Cherry, Black Currant, Cedar, Baking Spi...",5.0,3.0,5.0,2.0,4.0,3.0,3.0,...,3.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10
4,Carmenere,Red,"Raspberry, Bell Pepper, Black Plum, Paprika, V...",3.0,5.0,3.0,4.0,5.0,4.0,2.0,...,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1


#### We can visualize the number of mentioned wines and their classification using a pie chart
![Pie chart of wine classifications](./Img/WineMentionsClassification.png)

#### We will make an important assumption about our data. We will assume that when a wine is mentioned, it is regarded in a positive light. We can assume this because we are taking our data from the top posts of the subreddit, and we assume that the top posts aren't going to be about bashing a particular wine.

#### From that assumption, we take the flavor profiles and multiply their values with the count of wine mentions to see what flavors are preferred.

In [12]:
def distribute_count(row):
    row['Body'] = row['Body'] * row['Count']
    row['Red Fruit'] = row['Red Fruit'] * row['Count']
    row['Black Fruit'] = row['Black Fruit'] * row['Count']
    row['Floral'] = row['Floral'] * row['Count']
    row['Herbal'] = row['Herbal'] * row['Count']
    row['Pepper'] = row['Pepper'] * row['Count']
    row['Earth'] = row['Earth'] * row['Count']
    row['Baking Spice'] = row['Baking Spice'] * row['Count']
    row['Leather'] = row['Leather'] * row['Count']
    row['Astringency'] = row['Astringency'] * row['Count']
    row['Citrus Fruit'] = row['Citrus Fruit'] * row['Count']
    row['Stone Fruit'] = row['Stone Fruit'] * row['Count']
    row['Tropical Fruit'] = row['Tropical Fruit'] * row['Count']
    row['Honey'] = row['Honey'] * row['Count']
    row['Cream'] = row['Cream'] * row['Count']
    row['Minerality'] = row['Minerality'] * row['Count']
    row['Bitter'] = row['Bitter'] * row['Count']
    return row

# We multiply the values in the flavor columns of each wine by the number of times it is mentioned in our data
flavor_df = wines_df.apply(lambda x: distribute_count(x), axis=1)

# Lastly, we save our data as .csv and .xlsx files for visualization and possible later use
flavor_df.to_csv("Mentioned_Flavors.csv")
flavor_df.to_excel("Mentioned_Flavors.xlsx")

flavor_df.head()

,Wine Name,Classification,Flavors,Body,Red Fruit,Black Fruit,Floral,Herbal,Pepper,Earth,...,Leather,Astringency,Citrus Fruit,Stone Fruit,Tropical Fruit,Honey,Cream,Minerality,Bitter,Count
0,Albarino,White,"Lemon Zest, Grapefruit, Honeydew, Nectarine, S...",2.0,0.0,0.0,3.0,0.0,0.0,0.0,...,0.0,0.0,5.0,2.0,0.0,1.0,1.0,4.0,0.0,1
1,Barbera,Red,"Tart Cherry, Licorice, Blackberry, Dried Herbs...",6.0,10.0,4.0,2.0,10.0,6.0,10.0,...,4.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
2,Cabernet Franc,Red,"Strawberry, Raspberry, Bell Pepper, Crushed Gr...",3.0,5.0,0.0,2.0,5.0,2.0,5.0,...,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
3,Cabernet Sauvignon,Red,"Black Cherry, Black Currant, Cedar, Baking Spi...",50.0,30.0,50.0,20.0,40.0,30.0,30.0,...,30.0,40.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10
4,Carmenere,Red,"Raspberry, Bell Pepper, Black Plum, Paprika, V...",3.0,5.0,3.0,4.0,5.0,4.0,2.0,...,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1


#### We can visualize the most popular flavors using Tableau:
![Most popular wine flavors](./Img/CommonWineFlavors.png)

#### From this visual, we can see that the most common flavors are:
#### White Wines: Citrus Fruit, Stone Fruit, Body, and Floral
#### Red Wines: Baking Spice, Body, Earth, and Herbal

#### We can see from our wine_counts DataFrame that the most mentioned wine in our data is Champagne. Here, we will filter our data in order to track the mentions of champagne over time.

In [13]:
champagne_mentions_df = reddit_df[reddit_df['total_mentions'].isin([['Champagne']])]
print(champagne_mentions_df.head())

# Now, we will save this data for visualization
champagne_mentions_df.to_csv("Champagne_mentions.csv")
champagne_mentions_df.to_excel("Champagne_mentions.xlsx")

    subreddit                                              title selftext  \
0      r/wine  champagne and in-n-out to celebrate becoming a...      NaN   
22     r/wine  i'm an artist who just sold my *first* paintin...      NaN   
51     r/wine    getting in on that champagne and fried chicken!      NaN   
81     r/wine  it's my birthday and i only knew one way to ce...      NaN   
172    r/wine  opening a bottle of champagne with port wine t...      NaN   

     upvote_ratio     ups  downs   score          created_utc      id kind  \
0            0.97  2127.0    0.0  2127.0  2021-03-03 18:30:23  lxaj0v   t3   
22           0.99   899.0    0.0   899.0  2021-04-09 14:01:38  mnqlf7   t3   
51           0.95   729.0    0.0   729.0  2021-04-01 05:08:53  mhtw9m   t3   
81           0.98   624.0    0.0   624.0  2021-03-11 16:00:52  m34fe3   t3   
172          0.97   491.0    0.0   491.0  2019-01-20 16:33:11  ai4edr   t3   

    wines_mentioned_in_title wines_mentioned_in_selftext total_menti

#### Here is a visualization of the number mentions of Champage wine per month:
![Champagne mentions per month](./Img/ChampageMentionsPerMonth.png)

#### We can see that the most common month that Champagne is mentioned is in January, which is not surprising because one of the most popular American Holidays is New Years Day in where Champagne is a popular drink for this occasion.

#### [Here](https://public.tableau.com/app/profile/ron.l2728/viz/RedditWineProject/Dashboard1) is the full visual dashboard including these graphs along with interactive features! 

#### Thanks to [Winefolly](http://winefolly.com) for compiling these flavor profiles for all of these wines!